##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/capabilities/vision/video-understanding"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/docs/capabilities/vision/video-understanding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/google-gemini/gemma-cookbook/blob/main/docs/capabilities/vision/video-understanding.ipynb"><img src="https://www.kaggle.com/static/images/logos/kaggle-logo-transparent-300.png" height="32" width="70"/>Run in Kaggle</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fgoogle-gemini%2Fgemma-cookbook%2Fmain%2Fdocs%2Fcapabilities%2Fvision%2Fvideo-understanding.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/gemma-cookbook/blob/main/docs/capabilities/vision/video-understanding.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Video analysis with Gemma

Video data is a rich source of information that can help you accomplish tasks and understand the world around you. Using Gemma with video data can help you understand spacial relationships, interpret human interactions, and assist with situational awareness. This tutorial shows you how to get started processing video data with Gemma using Hugging Face Transformers. The Transformers Python library provides an API for accessing pre-trained generative AI models, including Gemma. For more information, see the [Transformers](https://huggingface.co/docs/transformers/en/index) documentation.


Note: Gemma 3 and later models support input of both images and text. Earlier versions of Gemma only support text input, except for some variants, including [PaliGemma](https://ai.google.dev/gemma/docs/setup).

## Setup

Before starting this tutorial, complete the following steps:

* Get access to Gemma by logging into [Hugging Face](https://huggingface.co/google/gemma-3-4b-pt) and selecting **Acknowledge license** for a Gemma model.
* Select a Colab runtime with sufficient resources to run
  the Gemma model size you want to run. [Learn more](https://ai.google.dev/gemma/docs/core#sizes).
* Generate a Hugging Face [Access Token](https://huggingface.co/docs/hub/en/security-tokens#how-to-manage-user-access-token) and add it to your Colab environment.

### Configure Access Token

Add your access token to Colab to enable downloading of Gemma models from the Hugging Face web site. Use the Colab **Secrets** feature to securely save your token without adding it to your working code.

To add your Hugging Face Access Token as a Secret:

1. Open the secrets tab by selecting the key icon on left side of the interface, or select **Tools > Command pallete**, type `secrets`, and press **Enter**.
2. Select **Add new secret** to add a new secret entry.
3. In the **Name** field, enter `HF_TOKEN`.
4. In the **Value** field, enter the text of your Hugging Face Access Token.
5. In the **Notebook access** field, select the switch to enable access.

Once you have entered your Access Token as `HF_TOKEN` and value, you can access the model:

### Install Python packages

Install the Hugging Face libraries required for running the Gemma model and making requests.

In [ ]:
# Install Pytorch & other libraries
%pip install "torch>=2.4.0"

# Install a transformers version that supports Gemma 3n (>= 4.53.0)
%pip install "transformers==4.53.3" huggingface-hub

Login into Hugging Face Hub.

In [2]:
# Login into Hugging Face Hub
from huggingface_hub import login
login()

## Prepare video data prompt

Gemma models interpret video data by breaking it down into it's component parts, specifically as images of video frames and audio clips extracted from the video file. The following code example shows how to extract frames and audio from a video file.

In [1]:
!wget https://github.com/bebechien/gemma/raw/refs/heads/main/videos/ForBiggerBlazes.mp4 -O /content/video.mp4
!mkdir /content/frames
# extract frames from video at 1 frame per second (video is 15 seconds)
!ffmpeg -i /content/video.mp4 -vf fps=1 /content/frames/%04d.jpg
# extract audio from video as a single, 15-second clip
!ffmpeg -i /content/video.mp4 -vn -c:a copy /content/audio.aac

--2026-02-24 03:29:06--  https://github.com/bebechien/gemma/raw/refs/heads/main/videos/ForBiggerBlazes.mp4
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bebechien/gemma/refs/heads/main/videos/ForBiggerBlazes.mp4 [following]
--2026-02-24 03:29:06--  https://raw.githubusercontent.com/bebechien/gemma/refs/heads/main/videos/ForBiggerBlazes.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2498125 (2.4M) [application/octet-stream]
Saving to: ‘/content/video.mp4’

/content/video.mp4  100%[===================>]   2.38M  --.-KB/s    in 0.06s   

2026-02-24 03:29:06 (37.2 MB/s) - ‘/content/video.mp4’

Once you have extracted frames and audio from your video file into separate image files and audio files, you can assemble those components into a prompt. The following code example shows how to add the extracted video data to a prompt structure, in preparation for prompting the model.

In [4]:
import os

content = []

for file in os.listdir("/content/frames"):
  content.append({"type": "image", "url": f"/content/frames/{file}"})

content.append({"type": "audio", "audio": "/content/audio.aac"})
content.append({"type": "text", "text": "What is shown in this video?"})

messages = [
    {
        "role": "user",
        "content": content
    },
]

For longer videos, you may need to reduce the number of frames per second captured to fit within Gemma model's context window. For audio clips, you should limit the total length of any audio clip to 30 seconds or less for best results.

## Configure model

When loading a Gemma model for use with audio data, configure the Transformer instance specifically for use with image and audio data. In particular, you must define a `processor` and `model` object using the `AutoProcessor` and `AutoModelForImageTextToText` classes, as shown in the following code example:

In [5]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText

GEMMA_MODEL_ID = "google/gemma-3n-E2B-it"

processor = AutoProcessor.from_pretrained(GEMMA_MODEL_ID, device_map="auto")
model = AutoModelForImageTextToText.from_pretrained(
            GEMMA_MODEL_ID, torch_dtype="auto", device_map="auto")

processor_config.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

## Run video data request

Once you have created a prompt with video data and configured the Gemma model `processor` and `model` objects, you can run the prompt to generate output. The following example code shows a request using a chat template, output generation, decoding of the response:

In [6]:
input_ids = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True, return_dict=True,
        return_tensors="pt",
)
input_ids = input_ids.to(model.device, dtype=model.dtype)

# Generate output from the model
outputs = model.generate(**input_ids, max_new_tokens=128)

# decode and print the output as text
text = processor.batch_decode(
    outputs,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)
print(text[0])

/usr/local/lib/python3.12/dist-packages/transformers/audio_utils.py:62: UserWarning: PySoundFile failed. Trying audioread instead.
  audio = librosa.load(audio, sr=sampling_rate)[0]
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (

<bos><start_of_turn>user


<start_of_image><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><image_soft_token><im

## Next steps

Build and explore more with Gemma models:

* [Inference with images and video](https://github.com/google-gemini/gemma-cookbook/blob/main/Gemma/%5BGemma_3%5DInference_images_and_videos.ipynb)
  in the Gemma Cookbook